In [136]:
import torch
from torch import nn
from torchvision import datasets
import fastai 
from torchvision.transforms import ToTensor
# from fastai.data.core import DataLoader
from torch.utils.data import DataLoader
from fastai.data.core import DataLoaders
from fastai.callback.core import Callback
from fastai.vision.all import Learner, Metric
from fastai import optimizer
import torch.nn.functional as F


In [3]:
# model = nn.Sequential(
#     nn.Linear(28*28, 30),
#     nn.ReLU(), 
#     nn.Linear(28*28, 10)
# )

In [137]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [138]:
batch_size = 256

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([256, 1, 28, 28])
Shape of y: torch.Size([256]) torch.int64


In [139]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [312]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(), 
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits 

model = NeuralNetwork().to(device)
model


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [141]:
# def loss_fn_custom(pred, y):
#     fn = nn.CrossEntropyLoss()
#     curr = fn(pred, y)

    

loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [142]:
class SGDBasic:
    def __init__(self, params, lr): 
        self.params,self.lr = list(params),lr
        self.state = {p: {} for p in self.params}
        self.hypers = [{'lr': lr}]
        
    def step(self, *args, **kwargs):
        for p in self.params: 
            p.data -= p.grad.data * self.lr

    def zero_grad(self, *args, **kwargs):
        for p in self.params:
            p.grad = None

    def set_hypers(self, **kwargs):
        if 'lr' in kwargs:
            self.lr = kwargs['lr']
            self.hypers[0]['lr'] = kwargs['lr']
            
# optimizer = BasicOptimizer(model.parameters(), lr=1e-3)

In [233]:

class ProxSGD:
    def __init__(self, params, lr):
        self.params, self.lr = list(params), lr
        self.state = {p: {} for p in self.params}
        self.hypers = [{'lr': lr}]
    
    def soft_threshold(self, x, eta):
        # Apply the soft-thresholding operator
        return F.softshrink(x, lambd=eta)
        
    def prox_operator(self, x, eta):
        # Use the soft-thresholding operator as the proximal step
        return self.soft_threshold(x, eta)

    def Gt(self, x, eta, x_grad):
        return (1/self.lr) * (x - self.prox_operator(x - self.lr * x_grad, eta))
        
    def step(self, *args, **kwargs):
        for p in self.params:
            if p.grad is not None:  # Ensure gradients exist
                p.data -= self.lr * self.Gt(p.data, 1e-5, p.grad.data)

    def zero_grad(self, *args, **kwargs):
        for p in self.params:
            p.grad = None

    def set_hypers(self, **kwargs):
        if 'lr' in kwargs:
            self.lr = kwargs['lr']
            self.hypers[0]['lr'] = kwargs['lr']


    

In [ ]:
class ProxSGDWithLinearSearch:
    def __init__(self, params, lr):
        self.params, self.lr = list(params), lr
        self.state = {p: {} for p in self.params}
        self.hypers = [{'lr': lr}]
        self.max_iter = 5
        self.eta = 1e-5
        
    def soft_threshold(self, x, eta):
        # Apply the soft-thresholding operator
        return F.softshrink(x, lambd=eta)
        
    def prox_operator(self, x):
        # Use the soft-thresholding operator as the proximal step
        return self.soft_threshold(x, self.eta)

    def Gt(self, x, step_size, x_grad):
        return (1/step_size) * (x - self.prox_operator(x - step_size * x_grad))
        
    def step(self, *args, **kwargs):
        model = kwargs.get("model")
        loss_fn = kwargs.get("loss_fn")
        X = kwargs.get("X")
        y = kwargs.get("y")
        
        orig_params = [p.data.clone() for p in self.params]
        step_size = self.lr
        with torch.no_grad():
            pred = model(X)
            old_loss = loss_fn(pred, y)
        
        for _ in range(self.max_iter):
            for p in self.params:
                if p.grad is not None: 
                    Gt_val = self.Gt(p.data, step_size, p.grad.data)
                    p.data = p.data - step_size * Gt_val
            with torch.no_grad():
                pred = model(X)
                new_loss = loss_fn(pred, y)
            if new_loss < old_loss:
                break
            else:
                for i, j in zip(self.params, orig_params):
                    i.data.copy_(j)
                step_size *= 0.5
        else: 
            for p in self.params:
                if p.grad is not None: 
                    Gt_val = self.Gt(p.data, step_size, p.grad.data)
                    p.data = p.data - step_size * Gt_val
        self.lr = step_size
    def zero_grad(self, *args, **kwargs):
        for p in self.params:
            p.grad = None

    def set_hypers(self, **kwargs):
        if 'lr' in kwargs:
            self.lr = kwargs['lr']
            self.hypers[0]['lr'] = kwargs['lr']


    

In [7]:
dls = DataLoaders(train_dataloader, test_dataloader)

In [8]:
class CustomAccuracy1(Metric):
    def __init__(self):
        self.correct = 0
        self.total = 0

    def reset(self):
        self.correct = 0
        self.total = 0

    def accumulate(self, learn):
        # Get max probability of the final dimension
        preds = learn.pred.argmax(dim=-1)
        self.correct += (preds == learn.y).sum().item()
        self.total += len(learn.y)

    @property
    def value(self):
        return (self.correct / self.total) * 100 if self.total > 0 else None

    @property
    def name(self):
        return "Accuracy"

In [9]:
class CustomAccuracy2(Metric):
    def __init__(self):
        self.correct = 0
        self.total = 0

    def reset(self):
        self.correct = 0
        self.total = 0

    def accumulate(self, learn):
        # Get max probability of the final dimension
        preds = learn.pred.argmax(dim=-1)
        self.correct += (preds == learn.y).sum().item()
        self.total += len(learn.y)

    @property
    def value(self):
        return (self.correct / self.total) * 100 if self.total > 0 else None

    @property
    def name(self):
        return "meowMeow2"

In [10]:
class MixAccuracy(Metric):
    def __init__(self):
        self.c1 = CustomAccuracy1()
        self.c2 = CustomAccuracy2()

    def reset(self):
        self.c1.reset()
        self.c2.reset()

    def accumulate(self, learn):
        self.c1.accumulate(learn)
        self.c2.accumulate(learn)

    @property
    def value(self):
        return self.c1.value, self.c2.value

    @property
    def name(self):
        return f"{self.c1.name}_{self.c2.name}"

In [15]:
class RegularizationCallback(Callback):
    # def __init__(self, model, lambda_reg):
    #     self.model = model
    #     self.lambda_reg = lambda_reg
    def __init__(self):
        self.lambda_reg = 1e-3
    def after_loss(self):
        # Compute L2 regularization term
        l1_reg = 0
        for param in self.model.parameters():
            l1_reg += torch.sum(torch.abs(param))
        # self.learn.loss += self.lambda_reg * l1_reg * self.lr
    def after_epoch(self):
        print(self.loss)
        
        

In [43]:
learner = Learner(dls, model, opt_func=ProxSGD, loss_func=loss_fn, metrics=CustomAccuracy1)

In [44]:
df = learner.fit(20, lr=1)
df

epoch,train_loss,valid_loss,Accuracy,time
0,0.263975,0.361188,89.190000,00:05
1,0.123027,0.197778,93.850000,00:05
2,0.081856,0.124340,96.210000,00:05
3,0.060845,0.102134,96.970000,00:05
4,0.046707,0.089291,97.370000,00:05
5,0.037000,0.090292,97.380000,00:05
6,1.404513,0.640888,81.900000,00:05
7,0.257546,0.395920,88.530000,00:05
8,0.192108,0.349512,89.250000,00:05
9,0.156875,0.226687,93.160000,00:05


In [45]:
s = 0
for param in model.parameters():
    s += torch.sum(torch.abs(param.data) <= .01) 
print(s)

tensor(545937, device='cuda:0')


In [77]:
s = 0
for param in model.parameters():
    s += torch.sum(torch.abs(param))
print(s)

tensor(14883.8662, device='cuda:0', grad_fn=<AddBackward0>)


In [62]:
m = nn.Threshold(0.1, 20)
input = torch.randn(2)
output = m(input)

In [64]:
output, input

(tensor([20.0000,  2.4018]), tensor([-1.2521,  2.4018]))

In [256]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step(model=model, loss_fn=loss_fn, X=X, y=y)
        optimizer.zero_grad()

        if batch % 100 == 0:
            print(f"BATCH: {batch} of {size/batch_size} batches")
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [144]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [318]:
for i in range(100):
    train(train_dataloader, model, loss_fn, ProxSGDWithLinearSearch(model.parameters(), 20))

BATCH: 0 of 234.375 batches
loss: 0.157250  [  256/60000]
BATCH: 100 of 234.375 batches
loss: 0.144504  [25856/60000]
BATCH: 200 of 234.375 batches
loss: 0.122185  [51456/60000]
BATCH: 0 of 234.375 batches
loss: 0.145666  [  256/60000]
BATCH: 100 of 234.375 batches
loss: 0.133919  [25856/60000]
BATCH: 200 of 234.375 batches
loss: 0.113675  [51456/60000]
BATCH: 0 of 234.375 batches
loss: 0.135665  [  256/60000]
BATCH: 100 of 234.375 batches
loss: 0.124232  [25856/60000]
BATCH: 200 of 234.375 batches
loss: 0.107301  [51456/60000]
BATCH: 0 of 234.375 batches
loss: 0.128515  [  256/60000]
BATCH: 100 of 234.375 batches
loss: 0.116555  [25856/60000]
BATCH: 200 of 234.375 batches
loss: 0.102810  [51456/60000]
BATCH: 0 of 234.375 batches
loss: 0.122287  [  256/60000]
BATCH: 100 of 234.375 batches
loss: 0.108769  [25856/60000]
BATCH: 200 of 234.375 batches
loss: 0.098999  [51456/60000]
BATCH: 0 of 234.375 batches
loss: 0.119193  [  256/60000]
BATCH: 100 of 234.375 batches
loss: 0.102385  [25856

In [319]:
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 98.0%, Avg loss: 0.065056 

